# Read Result

In [ ]:
import pandas as pd 
# wannaone
path = r'predicted_101_twit.xlsx'
input_data=pd.read_excel(r'predicted_101_twit.xlsx')

In [ ]:
channel = input_data['channel'].as_matrix()
## date = input_data['date'].as_matrix()
contents = input_data['contents'].as_matrix()
point = input_data['point'].as_matrix()
label = input_data['predict_label'].as_matrix()

In [ ]:
count_pos = len(input_data.loc[input_data['predict_label'] == 'POS'])
count_neu = len(input_data.loc[input_data['predict_label'] == 'NEU'])
count_neg = len(input_data.loc[input_data['predict_label'] == 'NEG'])
counts = [count_pos,count_neu,count_neg]
print(counts)

# Composing Data...

In [ ]:
import re
def ex_keyword(h_x,keyword):
    for i in range(len(h_x)):
        try:
            h_x[i] = h_x[i].replace(keyword,"")
        except:
            h_x[i] = str(h_x[i])
            pass
    return h_x

sub_contents = ex_keyword(contents,'워너원')
sub_contents = ex_keyword(sub_contents,'WANNAONE')

In [ ]:
# tokeinize - Twitter

from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Twitter
stopwords = ['Josa','PreEomi','Punctuation','Foreign','Alpha','Unknown','Eomi']

def tokenizer2 (doc):
    tagged = ['/'.join(t) for t in twit.pos(doc, norm=True, stem=True)]
    result_words = [word for word in tagged if word.split('/')[-1] == 'Noun' or 'Verb']
    result_nouns = [word.split('/')[0] for word in result_words if len(word.split('/')[0]) > 1]
    return result_nouns
 
vect2 = CountVectorizer(tokenizer=tokenizer2,ngram_range=(1,3),min_df=3)

## Classify Pos / Neg

In [ ]:
# point: 50 ~ 100
level_a = []
for i in range(len(sub_contents)):
    if point[i] >= 50:
        level_a.append(str(sub_contents[i]))

# point: -50 ~ 50
level_b = []
for i in range(len(sub_contents)):
    if point[i] >= -50 and point[i] < 50 :
        level_b.append(str(sub_contents[i]))

# point: -100 ~ -50
level_c = []
for i in range(len(sub_contents)):
    if point[i] < -50:
        level_c.append(str(sub_contents[i]))
doc_a = ''.join(level_a)
doc_b = ''.join(level_b)
doc_c = ''.join(level_c)
doc_all = [doc_a,doc_b,doc_c]

### Extract Nouns

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Twitter
twit = Twitter()
def tokenizer2 (doc):
    tagged = ['/'.join(t) for t in twit.pos(doc, norm=True, stem=True)]
    result_words = [word for word in tagged if word.split('/')[-1] == 'Noun']
    result_nouns = [word.split('/')[0] for word in result_words if len(word.split('/')[0]) > 1]
    return result_nouns

count_vect = CountVectorizer(tokenizer=tokenizer2, min_df=3)
count_all = count_vect.fit_transform(doc_all)

In [ ]:
count_a = count_vect.transform(level_a)
count_b = count_vect.transform(level_b)
count_c = count_vect.transform(level_c)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer().fit(count_all)
tf_all = tf_transformer.transform(count_all)
tf_a = tf_transformer.transform(count_a)
tf_b = tf_transformer.transform(count_b)
tf_c = tf_transformer.transform(count_c)

In [ ]:
tf_all_array = tf_all.toarray()
sort_index = tf_all_array.argsort(axis=1)
count_dict = count_vect.vocabulary_

for index in sort_index[0][-20:]:
    important_word = index
    key = list(count_dict.keys())[list(count_dict.values()).index(important_word)]
    print(key)

In [ ]:
count_dict = count_vect.vocabulary_
for index in sort_index[1][-20:]:
    important_word = index
    key = list(count_dict.keys())[list(count_dict.values()).index(important_word)]
    print(key)

In [ ]:
count_dict = count_vect.vocabulary_
for index in sort_index[2][-20:]:
    important_word = index
    key = list(count_dict.keys())[list(count_dict.values()).index(important_word)]
    print(key)

In [ ]:
tf_a.shape[0]

In [ ]:
def get_topics(topics, feature_names, sorting, topics_per_chunk=6, n_words=20):
    topic_sets = []
    for j in range(len(topics)):
        topic_sets.append([])
    # print top n_words frequent words
        for i in range(n_words):
            try:
                topic_sets[j].append(feature_names[sorting[j, i]])
            except:
                pass
    return topic_sets 

def doc_topics(topic_sets):
    doc_topics = []
    for i in range(len(topic_sets)):
        temp = ' '.join(topic_sets[i])
        doc_topics.append(temp)
    return doc_topics

# Level_A

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from math import log

for i in range(1,10):
    lda = LatentDirichletAllocation(n_topics=i, learning_method="batch", max_iter=10, random_state=0)
    document_topics = lda.fit_transform(tf_a)
    BIC = -tf_a.shape[1]*log(lda.perplexity(tf_a))-(i*(tf_a.shape[1]+1)/2)*log(tf_a.shape[0])
    print(i, lda.perplexity(tf_a), BIC)

## Topic Modeling - Positive

In [ ]:
# Recommend you to use the number of topics which has minimum value of perplexity

lda_p = LatentDirichletAllocation(n_topics=2, learning_method="batch", max_iter=20, random_state=0)
document_topics_p = lda_p.fit_transform(tf_a)

import numpy as np
sorting = np.argsort(lda_p.components_, axis=1)[:,::-1]
feature_names = np.array(count_vect.get_feature_names())

import mglearn
mglearn.tools.print_topics(topics=range(2), feature_names=feature_names, sorting=sorting, 
                           topics_per_chunk=5, n_words=20)

#pos_topic = get_topics(topics=range(5), feature_names=feature_names, sorting=sorting, topics_per_chunk=5, n_words=20)
#pos_topics = doc_topics(pos_topic)

In [ ]:
from wordcloud import WordCloud
from wordcloud import ImageColorGenerator
from PIL import Image
import numpy as np
import mglearn
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
color_pos = np.array(Image.open("blue.jpg"))
image_colors = ImageColorGenerator(color_pos)

wordcloud = WordCloud(font_path =r"C:\Windows\Fonts\NanumGothic.ttf",
                     relative_scaling = 0.2, background_color = 'white',
                      mask = color_pos,
                     min_font_size=1,max_font_size=40).generate(pos_topics[0])
plt.figure(figsize=(50,50))
plt.imshow(wordcloud.recolor(color_func=image_colors),interpolation='bilinear')
plt.axis("off")
plt.savefig('pos_topic.png')
plt.show()

# Level_B

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

for i in range(1,5):
    lda = LatentDirichletAllocation(n_topics=i, learning_method="batch", max_iter=10, random_state=0)
    document_topics = lda.fit_transform(tf_b)
    print(i, lda.perplexity(tf_b))

## Topic Modeling - Neutral

In [ ]:
# Recommend you to use the number of topics which has minimum value of perplexity

lda_p = LatentDirichletAllocation(n_topics=5, learning_method="batch", max_iter=20, random_state=0)
document_topics_p = lda_p.fit_transform(tf_b)

import numpy as np
sorting = np.argsort(lda_p.components_, axis=1)[:,::-1]
feature_names = np.array(count_vect.get_feature_names())

import mglearn
mglearn.tools.print_topics(topics=range(5), feature_names=feature_names, sorting=sorting, 
                           topics_per_chunk=5, n_words=20)

#neu_topic = get_topics(topics=range(5), feature_names=feature_names, sorting=sorting, topics_per_chunk=5, n_words=20)
#neu_topics = doc_topics(neu_topic)

In [ ]:
from wordcloud import WordCloud
from wordcloud import ImageColorGenerator
from PIL import Image
import numpy as np
import mglearn
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
color_neu = np.array(Image.open("yellow.jpg"))
image_colors = ImageColorGenerator(color_neu)

wordcloud = WordCloud(font_path =r"C:\Windows\Fonts\NanumGothic.ttf",
                     relative_scaling = 0.2, background_color = 'white',
                      mask = color_neu,
                     min_font_size=1,max_font_size=40).generate(neu_topics[0])
plt.figure(figsize=(50,50))
plt.imshow(wordcloud.recolor(color_func=image_colors),interpolation='bilinear')
plt.axis("off")
plt.savefig('neu_topic.png')
plt.show()

## Topic Modeling - Negative

In [ ]:
# Recommend you to use the number of topics which has minimum value of perplexity

lda_p = LatentDirichletAllocation(n_topics=5, learning_method="batch", max_iter=20, random_state=0)
document_topics_p = lda_p.fit_transform(tf_c)

import numpy as np
sorting = np.argsort(lda_p.components_, axis=1)[:,::-1]
feature_names = np.array(count_vect.get_feature_names())

import mglearn
mglearn.tools.print_topics(topics=range(5), feature_names=feature_names, sorting=sorting, 
                           topics_per_chunk=5, n_words=20)

neg_topic = get_topics(topics=range(5), feature_names=feature_names, sorting=sorting, topics_per_chunk=5, n_words=20)
neg_topics = doc_topics(neg_topic)

In [ ]:
from wordcloud import WordCloud
from wordcloud import ImageColorGenerator
from PIL import Image
import numpy as np
import mglearn
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
color_neg = np.array(Image.open("red.jpg"))
image_colors = ImageColorGenerator(color_neg)

wordcloud = WordCloud(font_path =r"C:\Windows\Fonts\NanumGothic.ttf",
                     relative_scaling = 0.2, background_color = 'white',
                      mask = color_neg,
                     min_font_size=1,max_font_size=40).generate(neg_topics[0])
plt.figure(figsize=(50,50))
plt.imshow(wordcloud.recolor(color_func=image_colors),interpolation='bilinear')
plt.axis("off")
plt.savefig('neg_topic.png')
plt.show()